### Import libraries

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import time
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split
import warnings
import re
from sklearn.model_selection import (train_test_split, GridSearchCV)
warnings.simplefilter(action='ignore', category=FutureWarning)

In [11]:
def main_inf(df):
    print("DataFrame shape: ", df.shape)
    print("Nan rate: ", df.isna().mean().mean())
    print("Doublons: ", df.duplicated().sum())  

### Import cleaned dataset

In [12]:
df =pd.read_csv('cleaned.csv') 
main_inf(df)
df.head()

DataFrame shape:  (356251, 177)
Nan rate:  0.08648233430669962
Doublons:  0


,Unnamed: 0,index,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,...,INSTAL_AMT_INSTALMENT_MEAN,INSTAL_AMT_INSTALMENT_SUM,INSTAL_AMT_PAYMENT_MIN,INSTAL_AMT_PAYMENT_MAX,INSTAL_AMT_PAYMENT_MEAN,INSTAL_AMT_PAYMENT_SUM,INSTAL_DAYS_ENTRY_PAYMENT_MAX,INSTAL_DAYS_ENTRY_PAYMENT_MEAN,INSTAL_DAYS_ENTRY_PAYMENT_SUM,INSTAL_COUNT
0,0,0.0,1.0,Cash loans,0.0,0.0,0.0,0.0,202500.0,406597.5,...,11559.247105,219625.695,9251.775,53093.745,11559.247105,219625.695,-49.0,-315.421053,-5993.0,19.0
1,1,1.0,0.0,Cash loans,1.0,0.0,1.0,0.0,270000.0,1293502.5,...,64754.586000,1618864.650,6662.970,560835.360,64754.586000,1618864.650,-544.0,-1385.320000,-34633.0,25.0
2,2,2.0,0.0,Revolving loans,0.0,1.0,0.0,0.0,67500.0,135000.0,...,7096.155000,21288.465,5357.250,10573.965,7096.155000,21288.465,-727.0,-761.666667,-2285.0,3.0
3,3,3.0,0.0,Cash loans,1.0,0.0,0.0,0.0,135000.0,312682.5,...,62947.088438,1007153.415,2482.920,691786.890,62947.088438,1007153.415,-12.0,-271.625000,-4346.0,16.0
4,4,4.0,0.0,Cash loans,0.0,0.0,0.0,0.0,121500.0,513000.0,...,12666.444545,835985.340,0.180,22678.785,12214.060227,806127.975,-14.0,-1032.242424,-68128.0,66.0


In [13]:
categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
df=df.drop(columns=categorical_columns)

In [14]:
train_df = df[df['TARGET'].notnull()]
X = np.array(train_df.drop(['TARGET'], axis=1))
y = train_df['TARGET'].values
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [15]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
from sklearn.model_selection import RandomizedSearchCV


# define the parameter distributions to search over
param_dist = {
    'max_depth': sp_randint(2, 20),
    'learning_rate': sp_uniform(0.001, 0.5),
    'n_estimators': sp_randint(10, 200),
    'num_leaves': sp_randint(10, 200),
    'min_child_samples': sp_randint(5, 100),
    'subsample': sp_uniform(0.1, 0.9),
    'colsample_bytree': sp_uniform(0.1, 0.9),
    'reg_alpha': sp_uniform(0.001, 10),
    'reg_lambda': sp_uniform(0.001, 10),
}

# create a LightGBM classifier instance
lgbm_clf = LGBMClassifier()

# create a randomized search object
rs = RandomizedSearchCV(
    estimator=lgbm_clf,
    param_distributions=param_dist,
    n_iter=50,
    cv=5,
    random_state=42,
    n_jobs=-1,
    scoring='accuracy', #"roc_auc"
)

# fit the randomized search object to the data
rs.fit(X_train, y_train)

# print the best parameters and score
print(f"Best parameters: {rs.best_params_}")
print(f"Best score: {rs.best_score_:.4f}")


Best parameters: {'colsample_bytree': 0.4069597159452327, 'learning_rate': 0.057736760620294536, 'max_depth': 11, 'min_child_samples': 11, 'n_estimators': 153, 'num_leaves': 99, 'reg_alpha': 9.822683433294355, 'reg_lambda': 5.167358912710143, 'subsample': 0.3347462573473681}
Best score: 0.9198
